### Making the Karpathy JSON format

In [23]:
import glob,os, shutil
import numpy as np

In [7]:
import pandas as pd
df = pd.DataFrame()

In [5]:
#Add caption folder path to extract all captions
caption_folder_path = 'captions/*.txt'

#Add image folder path to extract all images
image_folder_path = 'images/*.jpg'


image = []
caps = []
for i, j in zip(glob.glob(caption_folder_path), glob.glob(image_folder_path)):
    cap, img = os.path.split(i)[-1], os.path.split(j)[-1]
    
    #Add the correct folder name which contains the captions
    f = open('captions/'+ cap, "r")
    caps.append(f.read())
    image.append(img)
    

In [9]:
df['filename'] = image
df['raw_caps'] = caps
df['tokens']  = df['raw_caps'].apply(lambda x:x.split())

In [10]:
df.head()

,filename,raw_caps,tokens
0,00001.jpg,"By: K. Allen, PhD; A. Morhardt, PhD; T. Ritzma...","[By:, K., Allen,, PhD;, A., Morhardt,, PhD;, T..."
1,00002.jpg,"Materials: gloves, forceps, probe, scissors, s...","[Materials:, gloves,, forceps,, probe,, scisso..."
2,00003.jpg,Posterior view of pharynx in retropharyngeal s...,"[Posterior, view, of, pharynx, in, retropharyn..."
3,00004.jpg,Retropharyngeal space: potential space between...,"[Retropharyngeal, space:, potential, space, be..."
4,00005.jpg,Identify tip of greater horn of hyoid bone (ri...,"[Identify, tip, of, greater, horn, of, hyoid, ..."


In [11]:
import string
table = str.maketrans('', '', string.punctuation)

In [12]:
for i in range(len(df)):
    df['tokens'][i] = [w.translate(table) for w in df['tokens'][i]]
    df['tokens'][i] = [w for w in df['tokens'][i] if w]

In [24]:
from numpy.random import multinomial

n_total_samples = len(df)

indices = np.arange(n_total_samples)
inds_split = multinomial(n=1,
                         pvals=[0.8, 0.1, 0.1],
                         size=n_total_samples).argmax(axis=1)


In [26]:
df['set_label'] = inds_split

In [28]:
df['set_label'].value_counts()

0    11702
1     1449
2     1435
Name: set_label, dtype: int64

In [29]:
dataset = {}

In [30]:
dataset['images'] = {}
dataset['dataset'] = 'DAMICA'

In [31]:
dataset['images'] = [{"sentids":[i], "imgid":i, "sentences": [{"tokens": df['tokens'][i], "raw": df['raw_caps'][i], "imgid": i, "sentid": i}], 
                      "split": "full_dataset", "filename": df['filename'][i]} 
                     for i in range(df.shape[0])]

In [32]:
for i in range(len(dataset['images'])):
    if df['set_label'][i] == 2:
        dataset['images'][i]['split'] = "val"
    elif df['set_label'][i] == 1:
        dataset['images'][i]['split'] = "test"
    else:
        dataset['images'][i]['split'] = "train"

In [34]:
import json
with open('dataset_DAMICA.json', 'w') as outfile:
    json.dump(dataset, outfile)